<a href="https://colab.research.google.com/github/victormurcia/CTS_Test/blob/main/CTS_Parser_Development_Dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up the Environment

In [1]:
#Ensure that the required packages are installed in the current environment
!pip install numpy --quiet
!pip install pandas --quiet
!pip install spacy==3.4.4 --quiet
!pip install scispacy --quiet
!pip install medspacy --quiet
!pip install scispacy --quiet
!pip install negspacy --quiet
!pip install seaborn --quiet
!pip install matplotlib --quiet
!pip install "dask[complete]" --quiet
!pip install ipywidgets --quiet
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --quiet 
print('\n')

#Check the current build in Google Colab
!cat /etc/*release
print('\n')

#Check CUDA version
!nvcc --version



DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=20.04
DISTRIB_CODENAME=focal
DISTRIB_DESCRIPTION="Ubuntu 20.04.5 LTS"
NAME="Ubuntu"
VERSION="20.04.5 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04.5 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
#Import the required libraries/packages
#General utilities
import numpy as np
import pandas as pd
import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt
import os, random, time,sys
from ipywidgets import widgets, interact, interactive, fixed, interact_manual
from tqdm import tqdm

#NLP Stuff
import spacy
import scispacy
import medspacy
from medspacy.ner import TargetRule
from medspacy.visualization import visualize_ent
from scispacy.linking import EntityLinker
from scispacy.abbreviation import AbbreviationDetector
from scispacy.hyponym_detector import HyponymDetector
from negspacy.negation import Negex

#Enable data to be extracted from my Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load the Data

In [3]:
#Load the Synthetic Veteran Suicide Dataset from my Google Drive
data_dir = r'/content/drive/MyDrive/csv_usa_100k/' #Establish location of data
print('The data is located at:')
print(data_dir, '\n')

def find_csv_files(data_dir):
  """
  This function finds the .csv files located in data_dir and returns an alphabetically sorted file list.

  Parameters:
  data_dir (str) : Directory of files
  
  Returns:
  data_list (list) : list of .csv files
  """
  data_list = [f for f in os.listdir(data_dir) if f.endswith('.csv')] #List the .csv files
  print('The data files are:')
  data_list.sort() #Sort the .csv files alphabetically
  print(data_list,'\n')

  #Get names of .csv files without the extension for naming stuff later
  f_names = [s.replace(".csv","") for s in data_list]

  return data_list,f_names

#Get .csv files
data_list,f_names = find_csv_files(data_dir)

The data is located at:
/content/drive/MyDrive/csv_usa_100k/ 

The data files are:
['allergies.csv', 'careplans.csv', 'conditions.csv', 'devices.csv', 'encounters.csv', 'imaging_studies.csv', 'immunizations.csv', 'medications.csv', 'observations.csv', 'organizations.csv', 'patients.csv', 'payer_transitions.csv', 'payers.csv', 'procedures.csv', 'providers.csv', 'supplies.csv'] 



Some of these files have information that won't be useful for me so I'll remove them from my list now.  The files that I'll be removing are: encounters, organizations, payer_transitions, payers, and providers. 

In [4]:
files2remove = ['encounters.csv', 'organizations.csv', 'payer_transitions.csv', 'payers.csv', 'providers.csv'] 
def remove_files(data_list,files2remove):
  for f in data_list:
    if f in files2remove:
      data_list.remove(f)

  res = filter(lambda i: i not in files2remove, data_list)
  return list(res)

data_list = remove_files(data_list,files2remove)
print(data_list)

['allergies.csv', 'careplans.csv', 'conditions.csv', 'devices.csv', 'imaging_studies.csv', 'immunizations.csv', 'medications.csv', 'observations.csv', 'patients.csv', 'procedures.csv', 'supplies.csv']


In [5]:
#Add data path to beginning of each element of list so that I can easily access them later
def prepend(data_list, str):
    str += '% s'
    new_list = [str % i for i in data_list]
    return new_list

#Add 
csv_list = prepend(data_list,data_dir)
print('Full path of .csv files \n')
csv_list

Full path of .csv files 



['/content/drive/MyDrive/csv_usa_100k/allergies.csv',
 '/content/drive/MyDrive/csv_usa_100k/careplans.csv',
 '/content/drive/MyDrive/csv_usa_100k/conditions.csv',
 '/content/drive/MyDrive/csv_usa_100k/devices.csv',
 '/content/drive/MyDrive/csv_usa_100k/imaging_studies.csv',
 '/content/drive/MyDrive/csv_usa_100k/immunizations.csv',
 '/content/drive/MyDrive/csv_usa_100k/medications.csv',
 '/content/drive/MyDrive/csv_usa_100k/observations.csv',
 '/content/drive/MyDrive/csv_usa_100k/patients.csv',
 '/content/drive/MyDrive/csv_usa_100k/procedures.csv',
 '/content/drive/MyDrive/csv_usa_100k/supplies.csv']

## Create Dataframes With Dask

In [79]:
cols = list(pd.read_csv(csv_list[8], nrows =0)) #Only read the column headers
print(cols)

['Id', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP', 'LAT', 'LON', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE']


In [6]:
#After checking the contents of all the .csv files, I decided that the following columns across all .csv files will not be 
#included during loading
omit_cols = ['START','STOP','ENCOUNTER','BASE_COST','PAYER_COVERAGE','DATE','TYPE','DEATHDATE', 
             'SSN', 'DRIVERS', 'PASSPORT', 'MAIDEN','ADDRESS','FIRST', 'LAST', 'SUFFIX','START_YEAR', 
             'END_YEAR', 'PAYER','BASE_COST','QUANTITY']

In [7]:
def read_all_csvs(csv_list,omit_cols):
  """
  This function reads all the csv files in a list, loads each of them into a dask dataframe, and 
  then places them into a list called df_list that can be accessed later.

  Parameters:
  csv_list (list)  : List containing csv files
  omit_cols (list) : List containing names of columns to be omitted during dataframe loading

  Returns:
  df_list (list) :list containing dask dataframes
  """
  #Start timer
  start_time = time.time()

  #Initialize array that will contain the loaded dataframes
  df_list = []

  #Number of .csv files to load
  nFiles = len(csv_list)

  #Iterate through the csv files and load the dataframes
  for j in range(nFiles):
    cols    = list(pd.read_csv(csv_list[j], nrows = 0)) #Only read header row for column names
    temp_df = dd.read_csv(csv_list[j] , usecols =[i for i in cols if i not in omit_cols],assume_missing=True,dtype={'VALUE': 'object'}) #Make dask df
    df_list.append(temp_df)
  
  #End timer
  print("Loading of " + str(nFiles) + " .csv files took: %.2f seconds" % (time.time() - start_time))

  return df_list

df_list = read_all_csvs(csv_list,omit_cols)

Loading of 11 .csv files took: 2.93 seconds


## EDA

In [82]:
#Visualize descriptions columns of dataframe 
def make_countplots(df,name,n):

  if n < 25:
    val = n
  else:
    val = 25
  b = sns.countplot(data=df,y='DESCRIPTION',order = df['DESCRIPTION'].value_counts().index[:25])
  b.axes.set_title("Top "+ str(val) + ' ' + name.capitalize() + " Present in Dataset",fontsize=25)
  b.set_xlabel("Counts",fontsize=15)
  b.set_ylabel(name.capitalize(),fontsize=15)
  b.tick_params(labelsize=8)
  plt.show()

#View the number of unique patients in each dataframe
@interact
def show_nunique_vals_in_df(x=widgets.IntSlider(min=0,max=len(df_list)-1,step=1,value=0)):
  nPatients = df_list[x]['PATIENT'].nunique().compute()
  nDescriptions = df_list[x]['DESCRIPTION'].nunique().compute()
  print('The ' + f_names[x] + '.csv file has' ,nPatients, 'unique patients and',nDescriptions,'unique',f_names[x])
  make_countplots(df_list[x].compute(),f_names[x],nDescriptions)

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

## Restructuring and Merging Dataframes

In [8]:
def restructure_dfs(ref_df,source_df,nPatients):
    """
    This function combines the observations for a patient in the Synthetic Veteran Suicide Dataset into a 
    single row in a new dataframe. Doing this should help with the NLP processing later.
    
    Parameters
    ----------
    nPatients : int,  How many patients to process?
    ref_df    : df,   Dataframe containing restructured patient data (should be the first dataframe that was 
                      restructured)
    source_df : df,   Dataframe that contains information to be restructured
    ----------
    """
    # Create empty patient dataframe
    columns = source_df.columns
    temp_df = pd.DataFrame(index=np.arange(nPatients),columns=columns)    
    
    for i in range(ref_df.shape[0]):
        patient = ref_df['PATIENT'][i] #Patient ID    
        #dummy_df   = source_df[source_df['PATIENT'] == patient]
        
        #Programatically populate the dataframe by first making lists out of the columns and then inserting
        #these lists into cells in the resulting dataframe
        for k,col in enumerate(columns):
            if col == 'PATIENT':
                temp_df.at[i, 'PATIENT'] = patient    
            else:
                temp_df.at[i, col] = source_df[col].tolist()
                
    return temp_df

def make_patient_df(nPatients,df,df_list, s_list, init_rand=False):
    """
    This function combines the observations for a patient in the Synthetic Veteran Suicide Dataset into a 
    single row in a new dataframe. Doing this should help with the NLP processing later.
    
    Parameters
    ----------
    nPatients : int,  How many patients to process?
    df        : df,   Dataframe containing patient data
    df_list   : list, list of all dataframes in dataset
    init_rand : bool, Use random values to choose patients? Defaults to False so only first 10 patients are selected
    s_list    : list, List of strings to append to column names in the reformatted dataframes 
    ----------
    """

    #Start timer
    #start_time = time.time()

    #Determine number of unique patients in dataframe
    unique_patients = df['PATIENT'].nunique()
    
    #Initialize iterators and patient index
    i = 0; j = random.randrange(unique_patients)
    
    #Initialize index array
    index_list = []
    
    #Make list of restructured dataframes
    rs_df_list = []
    
    for i in range(nPatients):
      # Create empty patient dataframe
      columns = df.columns
      new_df  = pd.DataFrame(index=np.arange(nPatients),columns=columns)
      
      #Select a random patient from the existing dataframe (random sample)
      if init_rand == True:
        j = random.randrange(unique_patients)                
      else:
        j = i 
      
      #Check that patient id has not already been added
      if j not in index_list:
        index_list.append(j)
      else:
        while j in index_list:
            j = random.randrange(unique_patients) 

        index_list.append(j)
      
      #Subset the dataframe so as to only show the results associated with a given patient
      patient    = df['PATIENT'][j] #Patient ID
      dummy_df   = df[df['PATIENT'] == patient]
      
      #Programatically populate the dataframe by first making lists out of the columns and then inserting
      #these lists into cells in the resulting dataframe
      for k,col in enumerate(columns):
        if col == 'PATIENT':
            new_df.at[i, 'PATIENT'] = patient    
        else:
            new_df.at[i, col] = dummy_df[col].tolist()
      
      #Append the restructured dataframe to the list of dfs
      rs_df_list.append(new_df)

      #1. Select the source dataframe to be converted from df_list
      #2. Subset this dataframe so that only the rows with the current patient ids are selected
      for m,cdf in enumerate(df_list[1:]):
        columns = cdf.columns

        # Create empty patient dataframe
        new_df  = pd.DataFrame(index=np.arange(nPatients),columns=columns)

        if 'PATIENT' not in columns:
          dummy_df = cdf[cdf['Id'] == patient]
        else:
          dummy_df = cdf[cdf['PATIENT'] == patient]

        dummy_df = dummy_df.compute()

        for k,col in enumerate(columns):
          if 'PATIENT' not in columns:
            if col == 'Id':
              new_df.at[i, 'PATIENT'] = patient    
            else:
              new_df.at[i, col] = dummy_df[col].tolist()
          else:
            if col == 'PATIENT':
              new_df.at[i, 'PATIENT'] = patient    
            else:
              new_df.at[i, col] = dummy_df[col].tolist()

        rs_df_list.append(new_df)

    #Rename columns so as to have a suffix included for easy traceback to source file
    for m,df in enumerate(rs_df_list):
      columns = df.columns
      if 'PATIENT' not in columns:
        df.columns = [x + s_list[m] if x != 'Id' else x for x in columns]
      else:
        df.columns = [x + s_list[m] if x != 'PATIENT' else x for x in columns]    

    #print('These are the indices for the patient IDs used to generate the dataframe below. \n',index_list)

    #End timer
    #print("Reformatting dataframes took: %.2f seconds" % (time.time() - start_time))

    return rs_df_list  

s_list = ['_als','_cps','_cds','_dvs','_iss','_ims','_mds','_obs','_pts','_prs','_sps']
#rs_df_list  = make_patient_df(1, df_list[0].compute(),df_list, s_list, init_rand=True)

In [9]:
def patient_df_wrapper(nPatients):
  """
  This function simply iterates over the make_patient_df function so as to generate the restructured dataframes for each patient. It concatenates the resulting dataframes into a single dataframe that will be
  use for subsequent processing/analysis. 
  """

  print('Starting process...')

  #Start timer
  start_time = time.time()

  #Initialize array to contain dataframes
  multiple_patients_df_list = []

  #Generate the patient dataframes
  for i in tqdm(range(nPatients)):
    rs_df_list  = make_patient_df(1, df_list[0].compute(),df_list, s_list, init_rand=True)
    dfs = [df.set_index('PATIENT') for df in rs_df_list] #Set the index for each dataframe to be the PATIENT Id
    df = pd.concat(dfs, axis=1) #
    multiple_patients_df_list.append(df)
  
  df = pd.concat(multiple_patients_df_list, axis=0)

  #End timer
  print("\n Multipatient dataframe generated after: %.2f seconds" % (time.time() - start_time))

  return df

#Generate dataframe for 100 patients
multi_patient_df = patient_df_wrapper(1)

#Save resulting dataframe to a .csv file
multi_patient_df.to_csv('multi_veteran_df.csv')

#Visualize dataframe
multi_patient_df

Starting process...


100%|██████████| 1/1 [05:25<00:00, 325.25s/it]


 Multipatient dataframe generated after: 325.26 seconds


,CODE_als,DESCRIPTION_als,Id_cps,CODE_cps,DESCRIPTION_cps,REASONCODE_cps,REASONDESCRIPTION_cps,CODE_cds,DESCRIPTION_cds,CODE_dvs,...,LAT_pts,LON_pts,HEALTHCARE_EXPENSES_pts,HEALTHCARE_COVERAGE_pts,CODE_prs,DESCRIPTION_prs,REASONCODE_prs,REASONDESCRIPTION_prs,CODE_sps,DESCRIPTION_sps
PATIENT,,,,,,,,,,,,,,,,,,,,,
cc731c65-36f7-42db-8e47-b4c984eb28f1,"[419474003.0, 232350006.0, 232347008.0, 419263...","[Allergy to mould, House dust mite allergy, Da...","[cc3027a8-3395-48a5-adf9-6c9241b53cd7, 7bfda01...","[711282006.0, 384758001.0, 385691007.0, 539500...","[Skin condition care, Self-care interventions ...","[24079001.0, nan, 65966004.0, 10509002.0, 3984...","[Atopic dermatitis, nan, Fracture of forearm, ...","[24079001.0, 241929008.0, 232353008.0, 6536300...","[Atopic dermatitis, Acute allergic reaction, P...",[],...,[39.88579964718056],[-105.26456712250246],[1759845.04],[4856.58],"[430193006.0, 430193006.0, 313191000.0, 430193...","[Medication Reconciliation (procedure), Medica...","[nan, nan, nan, nan, nan, nan, 65966004.0, nan...","[nan, nan, nan, nan, nan, nan, Fracture of for...",[],[]


In [10]:
#I don't need the Id columns so I'll drop them
df2 = multi_patient_df[multi_patient_df.columns.drop(list(multi_patient_df.filter(regex='Id')))]
df2

,CODE_als,DESCRIPTION_als,CODE_cps,DESCRIPTION_cps,REASONCODE_cps,REASONDESCRIPTION_cps,CODE_cds,DESCRIPTION_cds,CODE_dvs,DESCRIPTION_dvs,...,LAT_pts,LON_pts,HEALTHCARE_EXPENSES_pts,HEALTHCARE_COVERAGE_pts,CODE_prs,DESCRIPTION_prs,REASONCODE_prs,REASONDESCRIPTION_prs,CODE_sps,DESCRIPTION_sps
PATIENT,,,,,,,,,,,,,,,,,,,,,
cc731c65-36f7-42db-8e47-b4c984eb28f1,"[419474003.0, 232350006.0, 232347008.0, 419263...","[Allergy to mould, House dust mite allergy, Da...","[711282006.0, 384758001.0, 385691007.0, 539500...","[Skin condition care, Self-care interventions ...","[24079001.0, nan, 65966004.0, 10509002.0, 3984...","[Atopic dermatitis, nan, Fracture of forearm, ...","[24079001.0, 241929008.0, 232353008.0, 6536300...","[Atopic dermatitis, Acute allergic reaction, P...",[],[],...,[39.88579964718056],[-105.26456712250246],[1759845.04],[4856.58],"[430193006.0, 430193006.0, 313191000.0, 430193...","[Medication Reconciliation (procedure), Medica...","[nan, nan, nan, nan, nan, nan, 65966004.0, nan...","[nan, nan, nan, nan, nan, nan, Fracture of for...",[],[]


In [11]:
#Rename columns to something more intuitive
new_names = ['AllergyCode',
 'Allergy',
 'CareplanCode',
 'Careplan',
 'CareplanReason',
 'CareplanReasonDescription',
 'ConditionCode',
 'Condition',
 'DeviceCode',
 'Device',
 'DeviceUDI',
 'ImageBodysiteCode',
 'ImageBodysiteDescription',
 'ImageModalityCode',
 'ImageModalityDescription',
 'ImageSOPCode',
 'ImageSOPDescription',
 'ImmunizationCode',
 'Immunization',
 'MedicationCode',
 'Medication',
 'MedicationDispenses',
 'MedicationCost',
 'MedicationReason',
 'MedicationReasonDescription',
 'ObservationCode',
 'Observation',
 'ObservationValue',
 'ObservationUnit',
 'PatientBirthday',
 'PatientPrefix',
 'PatientMarital',
 'PatientRace',
 'PatientEthnicity',
 'PatientGender',
 'PatientBirthplace',
 'PatientCity',
 'PatientState',
 'PatientCounty',
 'PatientZIP',
 'PatientLAT',
 'PatientLON',
 'PatientHEALTHCARE_EXPENSES',
 'PatientHEALTHCARE_COVERAGE',
 'ProcedureCode',
 'ProcedureDescription',
 'ProcedureReason',
 'ProcedureReasonDescription',
 'SupplyCode',
 'SupplyDescription']

df2.columns = new_names
df2

,AllergyCode,Allergy,CareplanCode,Careplan,CareplanReason,CareplanReasonDescription,ConditionCode,Condition,DeviceCode,Device,...,PatientLAT,PatientLON,PatientHEALTHCARE_EXPENSES,PatientHEALTHCARE_COVERAGE,ProcedureCode,ProcedureDescription,ProcedureReason,ProcedureReasonDescription,SupplyCode,SupplyDescription
PATIENT,,,,,,,,,,,,,,,,,,,,,
cc731c65-36f7-42db-8e47-b4c984eb28f1,"[419474003.0, 232350006.0, 232347008.0, 419263...","[Allergy to mould, House dust mite allergy, Da...","[711282006.0, 384758001.0, 385691007.0, 539500...","[Skin condition care, Self-care interventions ...","[24079001.0, nan, 65966004.0, 10509002.0, 3984...","[Atopic dermatitis, nan, Fracture of forearm, ...","[24079001.0, 241929008.0, 232353008.0, 6536300...","[Atopic dermatitis, Acute allergic reaction, P...",[],[],...,[39.88579964718056],[-105.26456712250246],[1759845.04],[4856.58],"[430193006.0, 430193006.0, 313191000.0, 430193...","[Medication Reconciliation (procedure), Medica...","[nan, nan, nan, nan, nan, nan, 65966004.0, nan...","[nan, nan, nan, nan, nan, nan, Fracture of for...",[],[]


In [12]:
test_str = ",".join(df2['Allergy'][0])

## Accessing Clinical Trials

In [13]:
#Query for trials
cond = 'allergy'#input('Enter the disease condition to find clinical trials: ')
a = 'https://clinicaltrials.gov/api/query/study_fields?expr='
b = '&fields=NCTId%2CBriefTitle%2CCondition%2COverallStatus%2CLeadSponsorName%2CEligibilityCriteria'
c = '&min_rnk=1&max_rnk=1000&fmt=csv'
q=(a + cond + b + c)
print(q)
qtrials = pd.read_csv(q,skiprows=10)

@interact
def show_recruiting_studies(column=['OverallStatus','Condition'], 
                            x = ['Recruiting','Completed','Unknown status'],
                            y = ['Food','Antibiotic']
                            ):
    if column == 'OverallStatus':
      return qtrials.loc[qtrials[column] == x]
    elif column == 'Condition':
      return qtrials[qtrials['Condition'].str.contains(y)]

https://clinicaltrials.gov/api/query/study_fields?expr=allergy&fields=NCTId%2CBriefTitle%2CCondition%2COverallStatus%2CLeadSponsorName%2CEligibilityCriteria&min_rnk=1&max_rnk=1000&fmt=csv


interactive(children=(Dropdown(description='column', options=('OverallStatus', 'Condition'), value='OverallSta…

In [14]:
qtrials

,Rank,NCTId,BriefTitle,Condition,OverallStatus,LeadSponsorName,EligibilityCriteria
0,1,NCT04827602,Drug Allergy Labels After Drug Allergy Investi...,Drug Hypersensitivity,Completed,"University Hospital, Gentofte, Copenhagen",Inclusion Criteria:||Penicillin allergy tested...
1,2,NCT03826953,Allergy UK Research and Development Nurse Project,Allergy,Unknown status,University of Edinburgh,"Inclusion Criteria:||All children, young peopl..."
2,3,NCT05561777,Penicillin Allergy Risk-Stratification and Del...,Antibiotic Allergy,Recruiting,Vanderbilt University Medical Center,Inclusion Criteria:||Pediatric Hospital Medici...
3,4,NCT01914978,Evaluating the Effectiveness of a Handbook for...,"Hypersensitivity, Food",Completed,Boston Children's Hospital,Inclusion Criteria:||Parents of children ages ...
4,5,NCT03581604,De-labeling of Patients With False Diagnosis o...,Allergy Drug,Recruiting,Oslo University Hospital,Inclusion Criteria:||Adult patients who are re...
...,...,...,...,...,...,...,...
995,996,NCT02378129,Evaluation of Dentin Hypersensitivity Using Gl...,Dentin Hypersensitivity,Completed,Federal University of Pelotas,Inclusion Criteria:||Male and female subjects ...
996,997,NCT02060864,Effect of AN-PEP Enzyme on Gluten Digestion in...,Non-coeliac Gluten Sensitivity,Completed,DSM Food Specialties,Inclusion Criteria:||Male/female|Age ≥18 but <...
997,998,NCT05704218,Hypersensitivity Pneumonitis of Domestic Origin,Sensitisation|Mold or Dust Allergy|Respiratory...,Not yet recruiting,Centre Hospitalier Universitaire de Besancon,Inclusion Criteria:||exposure to mold at home|...
998,999,NCT03703791,"Real World, Open Label, QOL Assessment of Pean...",Peanut Allergy,Terminated,"Aimmune Therapeutics, Inc.",Key Inclusion Criteria:||Age 4 through 17 year...


## NLP
Now that I have data for a patient consolidated I can start to parse it! There's a few different pretrained parser models that I can try fairly easily. I'll be making use of the spacy, medspacy, and scispacy libraries to aid with this. 

In [15]:
#I need to import locale to ensure that the
import locale
locale.getpreferredencoding = lambda: "UTF-8"
#I'll install spacy models used for processing biomedical, scientific, or clinical text 
#Spacy pipeline for biomedical data.
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz --quiet
#Spacy pipeline for biomedical data. Has a larger vocabulary and 50k word vectors
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_md-0.5.1.tar.gz --quiet
#This one is another spacy pipeline with 785k vocabulary and uses scibert-base as a transformer model
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_scibert-0.5.1.tar.gz --quiet
#Spacy pipeline for biomedical data with 600k word vectors
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_lg-0.5.1.tar.gz --quiet
#A spaCy NER model trained on the CRAFT corpus.
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_craft_md-0.5.1.tar.gz --quiet
#A spaCy NER model trained on the JNLPBA corpus.
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_jnlpba_md-0.5.1.tar.gz --quiet
#A spaCy NER model trained on the BC5CDR corpus.
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bc5cdr_md-0.5.1.tar.gz --quiet
#A spaCy NER model trained on the BIONLP13CG corpus.
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_ner_bionlp13cg_md-0.5.1.tar.gz --quiet
#This is the med7 transformer model found here: https://github.com/kormilitzin/med7
!pip install https://huggingface.co/kormilitzin/en_core_med7_trf/resolve/main/en_core_med7_trf-any-py3-none-any.whl --quiet
#This is the med7 vector model 
!pip install https://huggingface.co/kormilitzin/en_core_med7_lg/resolve/main/en_core_med7_lg-any-py3-none-any.whl --quiet

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 GB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.4/607.4 MB 2.4 MB/s eta 0:00:00


In [18]:
#Load the models
ss_sm            = spacy.load("en_core_sci_sm")
ss_md            = spacy.load("en_core_sci_md")
ss_bert          = spacy.load("en_core_sci_scibert")
ss_lg            = spacy.load("en_core_sci_lg")
ss_craft         = spacy.load("en_ner_craft_md")
ss_jnlpba        = spacy.load("en_ner_jnlpba_md")
ss_bionlp13cg_md = spacy.load("en_ner_bionlp13cg_md")
med7             = spacy.load("en_core_med7_lg")

In [30]:
med7.pipe_labels['ner']

['DOSAGE', 'DRUG', 'DURATION', 'FORM', 'FREQUENCY', 'ROUTE', 'STRENGTH']

In [31]:
# create distinct colours for labels
col_dict = {}
seven_colours = ['#e6194B', '#3cb44b', '#ffe119', '#ffd8b1', '#f58231', '#f032e6', '#42d4f4']
for label, colour in zip(med7.pipe_labels['ner'], seven_colours):
    col_dict[label] = colour

options = {'ents': med7.pipe_labels['ner'], 'colors':col_dict}


text = 'A patient was prescribed Magnesium hydroxide 400mg/5ml suspension PO of total 30ml bid for the next 5 days.'
doc = med7(text)

spacy.displacy.render(doc, style='ent', jupyter=True, options=options)

[(ent.text, ent.label_) for ent in doc.ents]

[('Magnesium hydroxide', 'DRUG'),
 ('400mg/5ml', 'DOSAGE'),
 ('suspension', 'FORM'),
 ('PO', 'ROUTE'),
 ('30ml', 'DOSAGE'),
 ('bid', 'FREQUENCY')]

In [27]:
#Select model to use in pipeline
nlp = spacy.load("en_core_sci_sm")

#Add the abbreviation detector to the pipeline
nlp.add_pipe("abbreviation_detector")

#Add the entity linker to the pipeline
nlp.add_pipe("scispacy_linker", config={"linker_name": "umls","max_entities_per_mention": 6})

#Add negation to the pipeline
nlp.add_pipe("negex")

https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2020-10-09/umls/tfidf_vectors_sparse.npz not found in cache, downloading to /tmp/tmp4ao2go3j
Finished download, copying /tmp/tmp4ao2go3j to cache at /root/.scispacy/datasets/e9f7327283e43f0482f7c0c71b71dec278a58ccb3ffdd03c2c2350159e7ef146.f2a350ad19015b2591545f7feeed6a6d6d2fffcd635d868a5d7fc0dfc3cadfd8.tfidf_vectors_sparse.npz
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2020-10-09/umls/nmslib_index.bin not found in cache, downloading to /tmp/tmpezexd8m0
Finished download, copying /tmp/tmpezexd8m0 to cache at /root/.scispacy/datasets/f48455d6c79262057cce66b4619123c2b558b21092d42fac97f47bb99a5b8f9f.dd70d3dffe7d90d7ac8914460e16a48375dab32485fb6313a34e6fbcaf53218b.nmslib_index.bin
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linkers/2020-10-09/umls/tfidf_vectorizer.joblib not found in cache, downloading to /tmp/tmppj7lw_u7
Finished download, copying /tmp/tmppj7lw_u7 to cache at /root/.scispacy/da

In [27]:
# Add the abbreviation pipe to the spacy pipeline.
doc = nlp("An allograft was used to recreate the coracoacromial ligaments and then secured to decorticate with a bioabsorbable tenodesis screw and then to the clavicle.")

fmt_str = "{:<15}| {:<6}| {:<7}| {:<8}"
print(fmt_str.format("token", "pos", "label", "parent"))

for token in doc:
    print(fmt_str.format(token.text, token.pos_, token.ent_type_, token.head.text))

token          | pos   | label  | parent  
An             | DET   |        | allograft
allograft      | NOUN  | ENTITY | used    
was            | AUX   |        | used    
used           | VERB  |        | used    
to             | PART  |        | recreate
recreate       | VERB  | ENTITY | used    
the            | DET   |        | ligaments
coracoacromial | ADJ   | ENTITY | ligaments
ligaments      | NOUN  | ENTITY | recreate
and            | CCONJ |        | recreate
then           | ADV   |        | secured 
secured        | VERB  |        | recreate
to             | PART  |        | decorticate
decorticate    | VERB  | ENTITY | secured 
with           | ADP   |        | tenodesis
a              | DET   |        | tenodesis
bioabsorbable  | ADJ   | ENTITY | tenodesis
tenodesis      | NOUN  | ENTITY | decorticate
screw          | VERB  | ENTITY | used    
and            | CCONJ |        | screw   
then           | ADV   |        | screw   
to             | ADP   |        | clavicle

In [29]:
#nlp.add_pipe("abbreviation_detector")

doc = nlp(
    "Chronic lymphocytic leukemia (CLL), autoimmune hemolytic anemia, and oral ulcer. The patient was diagnosed with chronic lymphocytic leukemia and was noted to have autoimmune hemolytic anemia at the time of his CLL diagnosis."
)

fmt_str = "{:<6}| {:<30}| {:<6}| {:<6}"
print(fmt_str.format("Short", "Long", "Starts", "Ends"))

for abrv in doc._.abbreviations:
    print(fmt_str.format(abrv.text, str(abrv._.long_form), abrv.start, abrv.end))

Short | Long                          | Starts| Ends  
CLL   | Chronic lymphocytic leukemia  | 4     | 5     
CLL   | Chronic lymphocytic leukemia  | 36    | 37    
